# Recursive Retriever + Node References

This guide shows how you can use recursive retrieval to traverse node relationships and fetch nodes based on "references".

Node references are a powerful concept. When you first perform retrieval, you may want to retrieve the reference as opposed to the raw text. You can have multiple references point to the same node.

In this guide we explore some different usages of node references:
- Different chunk sizes referring to a bigger chunk
- TBD

In [ ]:
!wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "data/llama2.pdf"

In [1]:
from pathlib import Path
from llama_hub.file.pdf.base import PDFReader

In [2]:
loader = PDFReader()
docs0 = loader.load_data(file=Path("./data/llama2.pdf"))

In [3]:
from llama_index import Document

doc_text = "\n\n".join([d.get_content() for d in docs0])
docs = [Document(text=doc_text)]

## Node References: Smaller Child Chunks Referring to Bigger Parent Chunk

In this usage example, we show how to build a graph of smaller chunks pointing to bigger parent chunks.

During query-time, we retrieve smaller chunks, but we follow references to bigger chunks. This allows us to have more context for synthesis.

In [4]:
from llama_index.node_parser import SimpleNodeParser
from llama_index.schema import IndexNode

In [5]:
node_parser = SimpleNodeParser.from_defaults(chunk_size=1024)

In [6]:
base_nodes = node_parser.get_nodes_from_documents(docs)

In [7]:
print(base_nodes[0].get_content())

Llama 2 : Open Foundation and Fine-Tuned Chat Models
Hugo Touvron∗Louis Martin†Kevin Stone†
Peter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra
Prajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen
Guillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller
Cynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou
Hakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev
Punit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich
Yinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar Mishra
Igor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein Rashi Rungta Kalyan Saladi
Alan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang
Ross Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang
Angela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic
Sergey Edunov 

In [15]:
sub_chunk_sizes = [128, 256, 512]
sub_node_parsers = [
    SimpleNodeParser.from_defaults(chunk_size=c) for c in sub_chunk_sizes
]

all_nodes = []
for base_node in base_nodes:
    for n in sub_node_parsers:
        sub_nodes = n.get_nodes_from_documents([base_node])
        sub_inodes = [
            IndexNode.from_text_node(sn, base_node.node_id) for sn in sub_nodes
        ]
        all_nodes.extend(sub_inodes)

    # also add original node to node
    base_inode = IndexNode.from_text_node(base_node, base_node.node_id)
    all_nodes.append(base_inode)

In [24]:
all_nodes_dict = {n.node_id: n for n in all_nodes}

In [17]:
# index in a vector db

## Load index into vector index
from llama_index import VectorStoreIndex, ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=llm)

vector_index = VectorStoreIndex(all_nodes, service_context=service_context)

In [18]:
vector_retriever = vector_index.as_retriever()

In [19]:
results = vector_retriever.retrieve(
    "Can you tell me about the key concepts for safety finetuning"
)

In [20]:
results

[NodeWithScore(node=IndexNode(id_='a0782caf-0763-4a73-b8ec-1035d223887d', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='4c404d8f-95dd-4205-b7c2-83ad151f9ed7', node_type=None, metadata={}, hash='e91a5d68ee74c66ce0e0c49fd32a5f6a17c4fbc26a6305919029a754f32c17d0'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='598481b3-fb73-4afc-bcc9-af8312494031', node_type=None, metadata={}, hash='968e147841df2e03a009747584987b0ccf284b1e79f467e5838fbfae38aa9f4a'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='51877384-dcca-4b70-9c3e-e9c0f16e0eb1', node_type=None, metadata={}, hash='1891e6c506d2ba476704c866e98906bea6621dd70f80213fa00ac3a8308fc597')}, hash='8adbc43ed86fb0e6e8727d401e4608bd66ef957246e865002bfb8efc1ff8d5b7', text='andthetechniquesweusetomitigatesafetyrisks.Weemployaprocesssimilartothegeneral\nfine-tuning methods as described in Section 3, with some nota

In [21]:
from llama_index.retrievers import RecursiveRetriever

In [25]:
recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    node_dict=all_nodes_dict,
    verbose=True,
)

In [29]:
nodes = recursive_retriever.retrieve(
    "Can you tell me about the key concepts for safety finetuning"
)
for node in nodes:
    print(node.node.get_content())

Retrieving with query id None: Can you tell me about the key concepts for safety finetuning
Retrieved node with id, entering: 4c404d8f-95dd-4205-b7c2-83ad151f9ed7
Retrieving with query id 4c404d8f-95dd-4205-b7c2-83ad151f9ed7: Can you tell me about the key concepts for safety finetuning
Retrieved node with id, entering: 61c5f66a-77e9-4c28-8310-028cb3461ed1
Retrieving with query id 61c5f66a-77e9-4c28-8310-028cb3461ed1: Can you tell me about the key concepts for safety finetuning
22

TruthfulQA ↑ToxiGen ↓
MPT7B 29.13 22.32
30B 35.25 22.61
Falcon7B 25.95 14.53
40B 40.39 23.44
Llama 17B 27.42 23.00
13B 41.74 23.08
33B 44.19 22.57
65B 48.71 21.77
Llama 27B 33.29 21.25
13B 41.86 26.10
34B 43.45 21.19
70B 50.18 24.60
Table 11: Evaluation of pretrained LLMs on automatic safety benchmarks.For TruthfulQA, we present the
percentageofgenerationsthatarebothtruthfulandinformative(thehigherthebetter).ForToxiGen,we
present the percentage of toxic generations (the smaller, the better).Benchmarks give a 